In [ ]:
import requests

#Replace with the server's actual IP and port
url = "http://127.0.0.1:8000/process"

#Path to the image you want to upload
image_file = "assets/framework.png"  # Replace with the path to your image file

#Open the image file and send it as part of a POST request
with open(image_file, "rb") as f:
    files = {"file": f}
    response = requests.post(url, files=files)

#Print the response from the server
if response.status_code == 200:
    print("Label:", response.json())
else:
    print("Failed to process image:", response.text)

In [ ]:
#DO NOT EDIT !!!

# import requests


# url = "http://127.0.0.1:8000/upload-video/"  # Replace <server-ip> with the server's IP
# #files = {'file': open('test.jpeg', 'rb')}  # Replace 'image.jpg' with your file path

# # response = requests.post(url, files=files)
# # if response.status_code == 200:
# #     print("Result:", response.json())
# # else:
# #     print("Error:", response.text)

# # Specify the video file to upload
# video_file = "clientVid2.mp4"  # Replace with your local video file path

# # Upload the video file
# with open(video_file, "rb") as f:
#     files = {"file": f}
#     response = requests.post(url, files=files)

# # Print the server's response
# if response.status_code == 200:
#     print("Upload successful:", response.json())
# else:
#     print("Upload failed:", response.json())

In [ ]:
!pip install fastapi

In [8]:
import torch
from torchvision import models, transforms
import os
import torch
from torch import nn
import numpy as np
from collections import OrderedDict
import torchvision.transforms as transforms
import torchvision.transforms._transforms_video as transforms_video
from moviepy.editor import *
#from transformers import AutoTokenizer
#from tqdm import tqdm
#import argparse

from src.data.video_transforms import Permute
from src.models.video_recap import VideoRecap
from src.data.datasets import VideoCaptionDataset, CaptionDataCollator
from src.models.timesformer import SpaceTimeTransformer
#from src.models.openai_model import QuickGELU
#from src.configs.defaults import defaultConfigs
#from PIL import Image
#import pickle
#from io import BytesIO

import requests
import base64
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse


In [9]:
# Directory to save uploaded videos
UPLOAD_DIRECTORY = "assets"
os.makedirs(UPLOAD_DIRECTORY, exist_ok=True)

# Specify the video file to upload
video_file = "clientVid1"  # Replace with your local video file path

# Save the uploaded video file
# file_path = os.path.join(UPLOAD_DIRECTORY, file.filename)
# with open(file_path, "wb") as f:
#     f.write(await file.read())
#vid = os.path.splitext(file.filename)[0]
vid = video_file #temporary solution, needs to be fixed
video_file = f'assets/{vid}.mp4'

In [13]:
# Create model and tokenizer
ckpt_path = 'pretrained_models/videorecap/videorecap_clip.pt'
ckpt = torch.load(ckpt_path, map_location='cpu')
old_args = ckpt['args']
old_args.video_feature_type = 'pixel'  
old_args.num_video_feat=4                     # number of frames per clip caption
crop_size = 224
transform = transforms.Compose([
        Permute([3, 0, 1, 2]),  # T H W C -> C T H W
        transforms.Resize(crop_size),
        transforms.CenterCrop(crop_size),
        transforms_video.NormalizeVideo(mean=[108.3272985, 116.7460125, 104.09373615000001], std=[68.5005327, 66.6321579, 70.32316305]),
    ])
#tokenizer = AutoTokenizer.from_pretrained(old_args.decoder_name)
state_dict = OrderedDict()
for k, v in ckpt['state_dict'].items():
    state_dict[k.replace('module.', '')] = v

print("=> Creating model")
model = VideoRecap(old_args, eval_only=True)
model = model.cuda()
model.load_state_dict(state_dict, strict=False)
print("=> loaded resume checkpoint '{}' (epoch {})".format(ckpt_path, ckpt['epoch']))


=> Creating model
######USING ATTENTION STYLE:  frozen-in-time


You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertLMHeadModel were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.l

Freeze the pretrained parts in Bert: ['bert.embeddings.word_embeddings.weight', 'bert.embeddings.position_embeddings.weight', 'bert.embeddings.LayerNorm.weight', 'bert.embeddings.LayerNorm.bias', 'bert.encoder.layer.0.attention.self.query.weight', 'bert.encoder.layer.0.attention.self.query.bias', 'bert.encoder.layer.0.attention.self.key.weight', 'bert.encoder.layer.0.attention.self.key.bias', 'bert.encoder.layer.0.attention.self.value.weight', 'bert.encoder.layer.0.attention.self.value.bias', 'bert.encoder.layer.0.attention.output.dense.weight', 'bert.encoder.layer.0.attention.output.dense.bias', 'bert.encoder.layer.0.attention.output.LayerNorm.weight', 'bert.encoder.layer.0.attention.output.LayerNorm.bias', 'bert.encoder.layer.0.intermediate.dense.weight', 'bert.encoder.layer.0.intermediate.dense.bias', 'bert.encoder.layer.0.output.dense.weight', 'bert.encoder.layer.0.output.dense.bias', 'bert.encoder.layer.0.output.LayerNorm.weight', 'bert.encoder.layer.0.output.LayerNorm.bias', 'ber

In [14]:
# Create dataset from the video
video = VideoFileClip(video_file)
print('Video length', video.duration, 'seconds')

video_length = video.duration
caption_duration = 4                              # Extract clip caption at each 4 seconds
old_args.video_loader_type='moviepy'
old_args.chunk_len = -1                           # load from raw video
old_args.video_feature_path = 'assets'            # path to the video folder 
metadata = []  
for i in np.arange(0, video_length, caption_duration):
    metadata.append([vid, i, min(i + caption_duration, video_length)])    # video name is example.mp4 so assuming video id=example
print('number of captions', len(metadata))
print(metadata)

old_args.metadata = metadata
dataset = VideoCaptionDataset(old_args, transform=transform)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=False, 
                                        num_workers=8, pin_memory=True, drop_last=False)
print(len(dataset), len(data_loader))
print(data_loader)

Video length 147.8 seconds
number of captions 37
[['clientVid1', 0.0, 4.0], ['clientVid1', 4.0, 8.0], ['clientVid1', 8.0, 12.0], ['clientVid1', 12.0, 16.0], ['clientVid1', 16.0, 20.0], ['clientVid1', 20.0, 24.0], ['clientVid1', 24.0, 28.0], ['clientVid1', 28.0, 32.0], ['clientVid1', 32.0, 36.0], ['clientVid1', 36.0, 40.0], ['clientVid1', 40.0, 44.0], ['clientVid1', 44.0, 48.0], ['clientVid1', 48.0, 52.0], ['clientVid1', 52.0, 56.0], ['clientVid1', 56.0, 60.0], ['clientVid1', 60.0, 64.0], ['clientVid1', 64.0, 68.0], ['clientVid1', 68.0, 72.0], ['clientVid1', 72.0, 76.0], ['clientVid1', 76.0, 80.0], ['clientVid1', 80.0, 84.0], ['clientVid1', 84.0, 88.0], ['clientVid1', 88.0, 92.0], ['clientVid1', 92.0, 96.0], ['clientVid1', 96.0, 100.0], ['clientVid1', 100.0, 104.0], ['clientVid1', 104.0, 108.0], ['clientVid1', 108.0, 112.0], ['clientVid1', 112.0, 116.0], ['clientVid1', 116.0, 120.0], ['clientVid1', 120.0, 124.0], ['clientVid1', 124.0, 128.0], ['clientVid1', 128.0, 132.0], ['clientVid1',

In [15]:
jj = 0
with torch.no_grad():
    for data_iter, samples in enumerate(data_loader):
        indices = samples['index']
        if hasattr(model, "vision_model"):
            image = samples["video_features"].permute(0, 2, 1, 3, 4).contiguous().cuda()  # BCTHW -> BTCHW
            samples["video_features"] = model.vision_model.forward_features(image, use_checkpoint=old_args.use_checkpoint, cls_at_last=False)  # NLD
            #print(samples["video_features"])
            
            # Get tensor and its shape
            tensor = samples["video_features"]
            tensor_shape = tensor.shape

            # Serialize the tensor to bytes and then encode it in base64 to make it JSON-compatible
            tensor_bytes = tensor.cpu().numpy().tobytes()
            tensor_base64 = base64.b64encode(tensor_bytes).decode('utf-8')  # base64 encoded string

            # Get tensor and its shape
            index_tensor = samples['index']
            index_tensor_shape = index_tensor.shape

            # Serialize the tensor to bytes and then encode it in base64 to make it JSON-compatible
            index_tensor_bytes = index_tensor.cpu().numpy().tobytes()
            index_tensor_base64 = base64.b64encode(index_tensor_bytes).decode('utf-8')  # base64 encoded string 
            
            # Prepare the data to send
            data = {
                "feature_tensor": tensor_base64,
                "feature_tensor_shape": list(tensor_shape),
                "index_tensor": index_tensor_base64,
                "index_tensor_shape": list(index_tensor_shape),
                "jj": jj
            }

            #Send the tensor and shape to the server in the body of a POST request
            response = requests.post(
                "http://127.0.0.1:8000/upload_tensor",
                json=data  # send as JSON body
            )

            print(response.json())
            jj+=1
        


#C C looks around
#C C walks around the house
#C C opens the door
#C C holds the cup with both hands
#C C drinks coffee
#C C walks on the field
#C C picks a bicycle
#C C looks around

#C C looks around
#C C looks around
#C C looks around
#C C rides the bicycle
#C C points at the man X
#C C points at the board
#C C passes the paper to his right hand with his left hand
#C C looks around

#C C looks around
#C C looks around
#C C climbs down the mountain
#C C takes a picture with the phone in her hands
#C C looks around the park
#C C looks around
#C C looks at the boat
#C C looks around

#C C looks around
#C C looks around
#C C opens the door
#C C looks around the house
#C C looks around
#C C looks around
#C C looks around
#C C mixes the seeds

#C C lights the fire
#C C looks around the fire
#C C looks around
#C C looks around
#C C looks at the mirror
